In [1]:
import pandas as pd
import gzip

def get_vcf_names(vcf_path):
    with gzip.open(vcf_path, "rt") as ifile:
          for line in ifile:
            if line.startswith("#CHROM"):
                  vcf_names = [x for x in line.split('\t')]
                  break
    ifile.close()
    return vcf_names


names = get_vcf_names('/home/kate/Documents/mnm/split_chr/CPCT02220079.annotated.processed_reheader_chr12_sorted.vcf.gz')
vcf = pd.read_csv('/home/kate/Documents/mnm/split_chr/CPCT02220079.annotated.processed_reheader_chr12_sorted.vcf.gz', compression='gzip', comment='#', delim_whitespace=True, header=None, names=names) #, chunksize=10000


In [2]:
vcf_pass = vcf.loc[vcf['FILTER'] == 'PASS']

In [3]:
vcf_pass

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,CPCT02220079R\n
0,12,60760,.,C,CA,24.75,PASS,AC=1;AF=0.5;AN=2;BaseQRankSum=0;ClippingRankSu...,GT:AD:DP:GQ:PL,"0/1:7,5:14:62:62,0,149"
2,12,61176,.,A,G,238.77,PASS,AC=1;AF=0.5;AN=2;BaseQRankSum=0;ClippingRankSu...,GT:AD:DP:GQ:PL,"0/1:37,11:48:99:267,0,1386"
3,12,61700,.,C,T,1811.77,PASS,AC=1;AF=0.5;AN=2;BaseQRankSum=-1.023;ClippingR...,GT:AD:DP:GQ:PL,"0/1:19,54:73:99:1840,0,574"
6,12,62122,rs138312253,A,C,406.77,PASS,AC=1;AF=0.5;AN=2;BaseQRankSum=1.37;ClippingRan...,GT:AD:DP:GQ:PGT:PID:PL,"0/1:23,13:36:99:0|1:62122_A_C:435,0,743"
7,12,62132,rs143808490,G,T,1388.77,PASS,AC=2;AF=1;AN=2;DB;DP=34;ExcessHet=3.0103;FS=0;...,GT:AD:DP:GQ:PGT:PID:PL,"1/1:0,34:34:99:1|1:62122_A_C:1417,105,0"
...,...,...,...,...,...,...,...,...,...,...
217430,12,133836499,rs140207272,A,G,2095.77,PASS,AC=2;AF=1;AN=2;DB;DP=51;ExcessHet=3.0103;FS=0;...,GT:AD:DP:GQ:PL,"1/1:0,51:51:99:2124,154,0"
217431,12,133840905,rs139789856,G,A,1636.77,PASS,AC=2;AF=1;AN=2;DB;DP=37;ExcessHet=3.0103;FS=0;...,GT:AD:DP:GQ:PGT:PID:PL,"1/1:0,37:37:99:1|1:133840905_G_A:1665,111,0"
217432,12,133840909,rs143158851,C,T,1677.77,PASS,AC=2;AF=1;AN=2;DB;DP=38;ExcessHet=3.0103;FS=0;...,GT:AD:DP:GQ:PGT:PID:PL,"1/1:0,38:38:99:1|1:133840905_G_A:1706,114,0"
217433,12,133841574,.,T,C,147.77,PASS,AC=1;AF=0.5;AN=2;BaseQRankSum=0.806;ClippingRa...,GT:AD:DP:GQ:PGT:PID:PL,"0/1:7,6:13:99:0|1:133841574_T_C:176,0,273"


In [4]:
af_poor = []
af_poor_all = []
af_all = []
poor_records = []

In [5]:
chromosomes = ["chr" + str(chr) for chr in range(1,23)]
chromosomes_add = ["chrMT", "chrX", "chrY"]
chrom = chromosomes + chromosomes_add

In [ ]:
for ch in chrom:
    #print('/home/kate/Documents/mnm/split_chr/CPCT02220079.annotated.processed_reheader_chr1_sorted.vcf.gz')
    names = get_vcf_names('/home/kate/Documents/mnm/split_chr/CPCT02220079.annotated.processed_reheader_' + ch + '_sorted.vcf.gz')
    vcf = pd.read_csv('/home/kate/Documents/mnm/split_chr/CPCT02220079.annotated.processed_reheader_' + ch + '_sorted.vcf.gz', compression='gzip', comment='#', delim_whitespace=True, header=None, names=names) #, chunksize=10000
    
    for i in range(len(vcf_pass['INFO'])):
        record = vcf_pass.iloc[i]
        record_string_all = record.INFO
           #print(record_string_all.split(";")[1])
        extracted_all = record_string_all.split(";")[1].split("=")[1] #Extract AF field
            #print(extracted_all)
        af_all.append(float(extracted_all)) #= list(af_all.append(extracted_all)
            #print(af_all)
        if '0/1' in record['CPCT02220079R\n']:
            if 'GoNLv5_AF' in record['INFO']:
                poor_records.append(record['INFO'])
                #print(record)
                record_string = record.INFO
                #print(record_string)
                #extracted = record_string.split(";")[1].split("=")[1] #extract AF field for poor heterozygotes
                #print(extracted)
                #af_poor_all.append(float(extracted))
                
                l = [line for line in record_string.split(";") if 'GoNLv5_AF' in line]
                extracted_pop_AF = float(l[0].split("=")[1])
                af_poor_all.append(extracted_pop_AF)
                
                if extracted_pop_AF<0.01:
                    #print(extracted)
                    af_poor.append(extracted_pop_AF)

In [ ]:
set(af_poor_all)

In [ ]:
set(af_poor)

In [ ]:
set(af_all)

In [ ]:
#Ratio 0/1 Allele Frequency = 0.5
af_poor_all.count(0.5)/len(af_all)

In [ ]:
#Ratio 0/1 Allele Frequency < 0.01
len(af_poor)/len(af_all)

In [47]:
poor_records[1].split(";")

['AC=1',
 'AF=0.5',
 'AN=2',
 'BaseQRankSum=1.46',
 'ClippingRankSum=0',
 'DB',
 'DP=102',
 'ExcessHet=3.0103',
 'FS=3.469',
 'MLEAC=1',
 'MLEAF=0.5',
 'MQ=40.24',
 'MQRankSum=-0.341',
 'QD=3.91',
 'ReadPosRankSum=-3.772',
 'SOR=0.35',
 'ANN=G|intron_variant|MODIFIER|ABC7-42389800N19.1|ENSG00000226210|transcript|ENST00000400706|unprocessed_pseudogene|2/10|n.292+2943C>G||||||',
 'GoNLv5_AC=430',
 'GoNLv5_AF=0.431',
 'GoNLv5_AN=998']

In [52]:
l = [line for line in poor_records[1].split(";") if 'GoNLv5_AF' in line]

In [56]:
float(l[0].split("=")[1])

0.431